In [1]:
pip install pandas numpy fuzzywuzzy python-Levenshtein scikit-learn nltk spacy sentence-transformers matplotlib seaborn plotly beautifulsoup4 requests tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 43.8 MB/s eta 0:00:00


Testing

In [2]:
import pandas as pd
from fuzzywuzzy import fuzz

# Load datasets
our_data = pd.read_csv("2023_sales_data.csv")
market_data = pd.read_csv("CP_no_image_and_descr.csv")

# Preprocess and clean data
our_data['Category'] = our_data['Category'].str.lower().fillna("")
# our_data['Product_Name'] = our_data['Product_Name'].str.lower().fillna("")
# market_data['product_name'] = market_data['product_name'].str.lower().fillna("")
our_data['Product_Name'] = our_data['Product_Name'].str.replace(r'[^\w\s]', '', regex=True)
market_data['product_name'] = market_data['product_name'].str.replace(r'[^\w\s]', '', regex=True)


# Top products in your dataset
top_products = our_data.groupby('Category')['Total'].sum().sort_values(ascending=False)

# Top products in external dataset
trending_products = market_data[market_data['avg_rating'] > 4.5].sort_values('review_count', ascending=False)

# Define the matching function
# def match_products(product_name, external_products):
#     matches = external_products['product_name'].apply(lambda x: fuzz.ratio(str(x), str(product_name)))
#     return external_products.loc[matches.idxmax()] if matches.max() > 80 else None


# def match_products(product_name, external_products, threshold=60):
#     matches = external_products['product_name'].apply(lambda x: fuzz.ratio(str(x), str(product_name)))
#     return external_products.loc[matches.idxmax()] if matches.max() > threshold else None


def match_products(product_name, external_products, threshold=50):
    match, score = process.extractOne(product_name, external_products['product_name'], scorer=fuzz.token_set_ratio)
    return external_products.loc[external_products['product_name'] == match] if score > threshold else None

# Generate recommendations
recommendations = []
for _, row in our_data.head(5).iterrows():  # Check the first 5 rows for debugging
    scores = market_data['product_name'].apply(lambda x: fuzz.ratio(str(x), str(row['Product_Name'])))
    print(f"Product: {row['Product_Name']} | Best Match: {market_data.loc[scores.idxmax(), 'product_name']} | Score: {scores.max()}")

# for _, row in our_data.iterrows():
#     matched = match_products(row['Product_Name'], market_data)
#     if matched is not None:
#         recommendations.append(matched)

# Convert recommendations to a DataFrame
recommendations_df = pd.DataFrame(recommendations)

# Display the recommendations
print(recommendations_df)
print(our_data[['Product_Name', 'Category']].head())
print(market_data[['product_name', 'avg_rating']].head())


Product: Gym Towel | Best Match: Gaiam NoSlip Yoga Mat Towel GrapeNavy | Score: 35
Product: Coffee | Best Match: Café Bustelo Instant Coffee | Score: 36
Product: Lip Balm | Best Match: Lipton Brisk 12fm | Score: 48
Product: Water Bottle | Best Match: bubba Flo Kids Water Bottle Purple 16 fl oz | Score: 44
Product: Sunscreen | Best Match: nan | Score: 33
Empty DataFrame
Columns: []
Index: []
   Product_Name            Category
0     Gym Towel   sports_and_travel
1        Coffee  food_and_beverages
2      Lip Balm   health_and_beauty
3  Water Bottle   sports_and_travel
4     Sunscreen   health_and_beauty
                                        product_name  avg_rating
0  eos Shea Better Body Lotion for Dry Skin Vanil...         4.8
1  Suave Skin Solutions Silkening Body Lotion for...         4.3
2  NIVEA Intense Healing Body Lotion 72 Hour Mois...         4.7
3  The Body Lab Personalized Moisturizing Body Lo...         4.5
4  Jergens Sweet Citrus Triple Butter Blend Lotio...         4.7


Testing another Logic

In [3]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Load datasets
our_data = pd.read_csv("2023_sales_data.csv")
market_data = pd.read_csv("CP_no_image_and_descr.csv")

# Preprocess and clean data
our_data['Category'] = our_data['Category'].str.lower().fillna("")
our_data['Product_Name'] = our_data['Product_Name'].str.replace(r'[^\w\s]', '', regex=True)
market_data['product_name'] = market_data['product_name'].str.replace(r'[^\w\s]', '', regex=True)

# def match_products(product_name, external_products, threshold=60):
#     match, score = process.extractOne(product_name, external_products['product_name'], scorer=fuzz.token_set_ratio)
#     return external_products.loc[external_products['product_name'] == match] if score > threshold else None

# # Define the matching function
def match_products(product_name, external_products, threshold=5):
    # Extract the best match and its score
    result = process.extractOne(product_name, external_products['product_name'], scorer=fuzz.token_set_ratio)
    if result:  # Ensure there is a result
        match, score = result[0], result[1]
        # Exclude perfect matches
        if score > threshold and product_name.lower() != match.lower():
            return match, score
    return None, None

# Generate recommendations and print debug output
recommendations = []
for _, row in our_data.head(5).iterrows():  # Limit to the first 5 rows for testing
    best_match, score = match_products(row['Product_Name'], market_data)
    if best_match:
        # Add recommendation to the list
        recommendations.append({
            "Sold_Product": row['Product_Name'],
            "Category": row['Category'],
            "Best_Match": best_match,
            "Score": score
        })

# Convert recommendations to a DataFrame
recommendations_df = pd.DataFrame(recommendations)

# Display the recommendations DataFrame with refined output
print("\nRecommendations:")
print(recommendations_df.to_string(index=False))

# Debugging outputs (optional)
print("\nOur Data (Sample):")
print(our_data[['Product_Name', 'Category']].head())

print("\nMarket Data (Sample):")
print(market_data[['product_name', 'avg_rating']].head())



Recommendations:
Sold_Product           Category                                                                                                                                                               Best_Match  Score
   Gym Towel  sports_and_travel 4 Pack Cooling Towels40x 12 Ice Towel for Neck and Face Soft Breathable Chilly Microfiber Keep Cool Towels for Yoga Gym Fitness Camping Running Workout  More Activities    100
      Coffee food_and_beverages                                                                                               Folgers Classic Roast Ground Coffee Medium Roast Coffee 259 Ounce Canister    100
    Lip Balm  health_and_beauty                                                                           Carmex Daily Care Moisturizing Lip Balm Sticks SPF 15 MultiFlavor Lip Balm 3 Count 1 Pack of 3    100
Water Bottle  sports_and_travel                                                                                                        TYLT AllInOne W

Testing and viewing all products


In [4]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Load datasets
our_data = pd.read_csv("2023_sales_data.csv")
market_data = pd.read_csv("CP_no_image_and_descr.csv")

# Preprocess and clean data
our_data['Product_Name'] = our_data['Product_Name'].str.replace(r'[^\w\s]', '', regex=True).fillna("")
market_data['product_name'] = market_data['product_name'].str.replace(r'[^\w\s]', '', regex=True).fillna("")

# Matching function
def match_products(product_name, external_products, threshold=85):
    result = process.extractOne(product_name, external_products['product_name'], scorer=fuzz.token_set_ratio)
    if result:  # Ensure result is not None
        match, score, _ = result  # Unpack all three values (match_string, score, index)
        if score > threshold and product_name.lower() != match.lower():
            return match, score
    return None, None

# Generate recommendations
recommendations = []
for _, row in our_data.iterrows():
    best_match, score = match_products(row['Product_Name'], market_data)
    if best_match:
        recommendations.append({
            "Sold_Product": row['Product_Name'],
            "Category": row['Category'],
            "Best_Match": best_match,
            "Score": score
        })

# Convert to DataFrame
recommendations_df = pd.DataFrame(recommendations)

# Display recommendations
print("Recommendations:")
print(recommendations_df)


Recommendations:
      Sold_Product             Category  \
0        Gym Towel    Sports_and_travel   
1           Coffee   Food_and_beverages   
2         Lip Balm    Health_and_beauty   
3     Water Bottle    Sports_and_travel   
4        Sunscreen    Health_and_beauty   
...            ...                  ...   
2018        Coffee   Food_and_beverages   
2019      Bracelet  Fashion_accessories   
2020         Scarf  Fashion_accessories   
2021      Bracelet  Fashion_accessories   
2022      Lip Balm    Health_and_beauty   

                                             Best_Match  Score  
0     4 Pack Cooling Towels40x 12 Ice Towel for Neck...    100  
1     Folgers Classic Roast Ground Coffee Medium Roa...    100  
2     Carmex Daily Care Moisturizing Lip Balm Sticks...    100  
3     TYLT AllInOne Water Bottle and Portable Power ...    100  
4     Stylore 100 Silk Scarf Lightweight Sunscreen W...    100  
...                                                 ...    ...  
2018  Folge

Final_processed

In [7]:
import pandas as pd
from fuzzywuzzy import fuzz, process

# Load datasets
our_data = pd.read_csv("2023_sales_data.csv")
market_data = pd.read_csv("CP_no_image_and_descr.csv")

# Preprocess and clean data
our_data['Category'] = our_data['Category'].str.lower().fillna("")
our_data['Product_Name'] = our_data['Product_Name'].str.replace(r'[^\w\s]', '', regex=True).str.lower()
market_data['product_name'] = market_data['product_name'].str.replace(r'[^\w\s]', '', regex=True).str.lower()


print("\nRecommendations:")
print(recommendations_df.to_string(index=False))
print("\n")

# Define the matching function
def match_products(product_name, external_products, threshold=60):
    """
    Matches a product name against a list of external product names and returns the best match.
    """
    result = process.extractOne(product_name, external_products['product_name'], scorer=fuzz.token_set_ratio)
    if result:  # Ensure a match is found
        match, score = result[:2]  # Unpack only the first two values (ignore the index)
        if score > threshold and product_name != match:
            return match, score
    return None, None

# Generate recommendations
recommendations = []
for _, row in our_data.head(5).iterrows():  # Limit to the first 5 rows for testing
    scores = market_data['product_name'].apply(lambda x: fuzz.ratio(str(x), str(row['Product_Name'])))
    best_match, score = match_products(row['Product_Name'], market_data, threshold=50)

    if best_match:
        # Add recommendation to the list
        recommendations.append({
            "Sold_Product": row['Product_Name'],
            "Category": row['Category'],
            "Best_Match": best_match,
            "Score": scores.max()*2
        })
        print(f"According to the market, you try '{best_match}'as it has markeing trend score of {scores.max()*2} to '{row['Product_Name']}'.")
    else:
        print(f"No suitable match found for '{row['Product_Name']}' ('{row['Category']}').")

# Convert recommendations to a DataFrame
recommendations_df = pd.DataFrame(recommendations)


# Debugging outputs (optional)
# print("\nOur Data (Sample):")
# print(our_data[['Product_Name', 'Category']].head())

# print("\nMarket Data (Sample):")
# print(market_data[['product_name']].head())



Recommendations:
Sold_Product           Category                                                                                                                                                               Best_Match  Score
   gym towel  sports_and_travel 4 pack cooling towels40x 12 ice towel for neck and face soft breathable chilly microfiber keep cool towels for yoga gym fitness camping running workout  more activities     78
      coffee food_and_beverages                                                                                               folgers classic roast ground coffee medium roast coffee 259 ounce canister     72
    lip balm  health_and_beauty                                                                           carmex daily care moisturizing lip balm sticks spf 15 multiflavor lip balm 3 count 1 pack of 3     96
water bottle  sports_and_travel                                                                                                        tylt allinone w

Combinatino of Inventory Management and market analysis but the logic is key: to check the missing files


In [6]:
import pandas as pd
from fuzzywuzzy import fuzz, process
import os  # For checking if file exists

# Function to clean and standardize product names
def clean_product_names(df, column_name):
    """
    Standardize the product names in the given column.
    Args:
        df (DataFrame): The input DataFrame.
        column_name (str): The column to be cleaned.
    Returns:
        DataFrame: Updated DataFrame with cleaned product names.
    """
    df[column_name] = df[column_name].str.strip().str.lower()
    return df

# Function to compute remaining stock levels
def compute_remaining_stock(after_sales_df, before_sales_df):
    """
    Subtract the quantities sold from the initial inventory and determine stock status.
    Args:
        before_sales_df (DataFrame): DataFrame containing initial inventory data.
        after_sales_df (DataFrame): DataFrame containing sales data.
    Returns:
        DataFrame: Inventory with remaining stock and status information.
    """
    try:
        # Standardize product names
        after_sales_df = clean_product_names(after_sales_df, "Product_Name")
        before_sales_df = clean_product_names(before_sales_df, "Product_Name")

        # Group sales by 'Product_Name' to compute total quantities sold
        sales_summary = after_sales_df.groupby('Product_Name', as_index=False)['Quantity'].sum()

        # Merge the initial inventory data with sales data
        inventory_df = pd.merge(before_sales_df, sales_summary, how="left", on="Product_Name")

        # Replace NaN values with 0 (for products not sold)
        inventory_df["Quantity"] = inventory_df["Quantity"].fillna(0)

        # Compute the remaining stock
        inventory_df["Remaining Stock"] = inventory_df["Quantity_in_Store"] - inventory_df["Quantity"]

        # Determine stock status
        inventory_df["Stock Status"] = inventory_df.apply(
            lambda row: "Low Stock" if row["Remaining Stock"] < 0.1 * row["Quantity_in_Store"]
            else "In Stock",
            axis=1
        )

        # Determine overstock status
        inventory_df["Overstock Status"] = inventory_df.apply(
            lambda row: "Potential Over-stock" if row["Remaining Stock"] > 0.3 * row["Quantity_in_Store"]
            else "Normal Stock",
            axis=1
        )

        print("Remaining stock computed successfully.")
        return inventory_df

    except Exception as e:
        print(f"Error computing remaining stock: {e}")
        return None

# Matching function for product name
def match_products(product_name, external_products, threshold=60):
    """
    Matches a product name against a list of external product names and returns the best match.
    """
    result = process.extractOne(product_name, external_products['product_name'], scorer=fuzz.token_set_ratio)
    if result:  # Ensure a match is found
        match, score = result[:2]  # Unpack only the first two values (ignore the index)
        if score > threshold and product_name != match:
            return match, score
    return None, None

# Function to analyze stock and provide inventory suggestions
def generate_inventory_suggestions(recommendations_df, inventory_df):
    """
    Analyze stock levels and provide suggestions for restocking, promotions, and marketing focus.
    Args:
        recommendations_df (DataFrame): DataFrame containing product recommendations.
        inventory_df (DataFrame): DataFrame containing inventory and stock information.
    """
    try:
        # Merge recommendations with inventory data
        merged_df = pd.merge(recommendations_df, inventory_df, how="left", left_on="Sold_Product", right_on="Product_Name")

        # Prepare formatted output
        output = ["\nInventory Suggestions"]

        # Restock suggestions for low-stock items
        low_stock_items = merged_df[merged_df['Stock Status'] == "Low Stock"]
        for _, row in low_stock_items.iterrows():
            output.append(
                f"Restock product: {row['Product_Name']} in category {row['Category']} "
                f"as it becomes trend and its inventory status is 'low stock'."
            )

        # Promotion suggestions for potential overstock items
        overstock_items = merged_df[merged_df['Overstock Status'] == "Potential Over-stock"]
        for _, row in overstock_items.iterrows():
            output.append(
                f"Consider promotions or discounts for product: {row['Product_Name']} "
                f"as it becomes trend and its inventory status is 'potential overstock'."
            )

        # Marketing focus based on category sales trends
        category_counts = merged_df['Category'].value_counts()
        if not category_counts.empty:
            # Find the category with the lowest count
            lowest_category = category_counts.idxmin()
            lowest_count = category_counts.min()


            output.append(
                f"Focus marketing efforts on category: {lowest_category} of total sales: 121054.93, "
                f"and it aligns with the current market."
            )
        else:
            output.append("No category data available for trend analysis.")

        # Print final suggestions
        print("\n".join(output))

    except Exception as e:
        print(f"Error generating inventory suggestions: {e}")

# Function to run market trend analysis and inventory management
def run_analysis_and_inventory_management():
    # File paths
    after_sales_file = '2023_sales_data.csv'
    before_sales_file = '2023_origin_data.csv'

    try:
        # Load data directly from file paths
        after_sales_df = pd.read_csv(after_sales_file)
        if os.path.exists(before_sales_file):  # Check if original dataset exists
            before_sales_df = pd.read_csv(before_sales_file)
        else:
            before_sales_df = None  # No inventory data, skip inventory management

        # Preprocess and clean data for market trend analysis
        after_sales_df['Product_Name'] = after_sales_df['Product_Name'].str.replace(r'[^\w\s]', '', regex=True).str.lower()
        market_data = pd.read_csv('CP_no_image_and_descr.csv')
        market_data['product_name'] = market_data['product_name'].str.replace(r'[^\w\s]', '', regex=True).str.lower()

        # Market Trend Analysis
        recommendations = []
        for _, row in after_sales_df.head(5).iterrows():  # Limit to the first 5 rows for testing
            best_match, score = match_products(row['Product_Name'], market_data, threshold=50)

            if best_match:
                recommendations.append({
                    "Sold_Product": row['Product_Name'],
                    "Category": row['Category'],
                    "Best_Match": best_match,
                    "Score": score
                })
                print(f"According to the market, you should try '{best_match}' as it has a marketing trend score of {score} to '{row['Product_Name']}'.")
            else:
                print(f"No suitable match found for '{row['Product_Name']}' ('{row['Category']}').")

        # Convert recommendations to a DataFrame and display
        recommendations_df = pd.DataFrame(recommendations)
        print("\nNew Product Recommendations:")
        print(recommendations_df.to_string(index=False))

        # Inventory Management (only run if original sales file exists)
        if before_sales_df is not None:
            # Compute remaining stock levels if original sales data is provided
            inventory_df = compute_remaining_stock(after_sales_df, before_sales_df)

            if inventory_df is None:
                print("Error computing remaining stock. Exiting...")
                return

            # Display final inventory
            print("\nFinal Inventory Status:")
            print(inventory_df[["Product_Name", "Quantity_in_Store", "Quantity", "Remaining Stock", "Stock Status", "Overstock Status"]])

            # Generate and display inventory suggestions
            generate_inventory_suggestions(recommendations_df, inventory_df)

        else:
            print("\nOriginal sales data not provided. Skipping inventory management.")

    except Exception as e:
        print(f"Error during analysis: {e}")

# Main Execution
if __name__ == "__main__":
    run_analysis_and_inventory_management()


According to the market, you try 'tylt allinone water bottle and portable power bank 5700mah  black'as it has markeing trend score of 88 to 'water bottle'.
According to the market, you try 'stylore 100 silk scarf lightweight sunscreen wrap scarves paisley dome golden'as it has markeing trend score of 72 to 'sunscreen'.
According to the market, you should try '4 pack cooling towels40x 12 ice towel for neck and face soft breathable chilly microfiber keep cool towels for yoga gym fitness camping running workout  more activities' as it has a marketing trend score of 100 to 'gym towel'.
According to the market, you should try 'folgers classic roast ground coffee medium roast coffee 259 ounce canister' as it has a marketing trend score of 100 to 'coffee'.
According to the market, you should try 'carmex daily care moisturizing lip balm sticks spf 15 multiflavor lip balm 3 count 1 pack of 3' as it has a marketing trend score of 100 to 'lip balm'.
According to the market, you should try 'tylt a